# Site API Design
<br><br>
>* [Create a Site Transaction API](#Create-a-Site-Transaction-API)
<br><br>
>* [Site Transaction Heartbeat API](#Site-Transaction-Heartbeat-API)
<br><br>
>* [Commit Site Transaction API](#Commit-Site-Transaction-API)
<br><br>
>* [Remove Site Transaction API](#Remove-Site-Transaction-API)
<br><br>
>* [Create Sites API](#Create-Sites-API)
<br><br>
>* [Create a Parent Site API](#Create-a-Parent-Site-API)
<br><br>
>* [Create a Leaf Site API](#Create-a-Leaf-Site-API)
<br><br>
>* [Delete a Site API](#Delete-a-Site-API)
<br><br>
>* [Get Site Info API](#Get-Site-Info-API)
<br><br>
>* [Get Child Sites API](#Get-Child-Sites-API)
<br><br>
>* [Add Site Devices API](#Add-Site-Devices-API)
<br><br>
>* [Delete Site Devices API](#Delete-Site-Devices-API)
<br><br>
>* [Replace Site Devices API](#Replace-Site-Devices-API)
<br><br>
>* [Get Site Devices API](#Get-Site-Devices-API)

# Create a Site Transaction API
All site modification operations must be executed in a transaction. In another word, the user should create a transaction before starting any other site changes for example, create site, move devices.
<br><br>
And also, after change site, the user should explicitly commit the operations . 
<br><br>
Note that  a site transaction will lock the entire NetBrain system for site change operations. To prevent a system-wide dead lock due to client exception or negligence,  if no follow-up operations or heartbeat sent within a 30 seconds time frame, another could invalidate this transaction, and create a new transaction which cannot used by the current session.
<br><br>
Deleting a transaction could let the user  to discard any site change operations since the beginning of a transaction, or  called rollback.

<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **POST** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Sites/Transactions | Yes |

<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |

<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|nb_URL* | string  | IP address of your NetBrain Web API Server  |

<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|statusCode| integer | Code issued by NetBrain server indicating the execution result.<br>790200 for success.<br>792102 if site locked by other operations. |
|statusDescription| string | The explanation of the status code. |

<br><br>

* **Example**

In [1]:
# import python modules 
import requests
import time
import urllib3
import pprint
import json
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set the request parameters
token = "b809ab0b-6ad3-4208-8571-f22f68b55860"
nb_url = "http://192.168.28.79"

# All site modification operations must be executed in a transaction
def createSiteTransaction(nb_url, token):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    full_url= nb_url + "/ServicesAPI/API/V1/CMDB/Sites/Transactions"
    try:
        response = requests.post(full_url, headers=headers, verify=False)
        if response.status_code == 200:
            result = response.json()
            return (result)       
        else:
            return ("Site transaction create Failed! - " + str(response.text))
    except Exception as e:
        return (str(e))

res = createSiteTransaction(nb_url, token)
res

'Site transaction create Failed! - {"statusCode":795003,"statusDescription":"Insufficient permissions: the current user has insufficient permissions to perform the requested operation. The user has no tenant or domain access permission.siteManagement"}'

# Site Transaction Heartbeat API
This API send a hearbeat signal to the server to keep a transaction alive.
<br><br>
Failed to do so will cause transaction being disgarded by the system if no other site change operations sent to the server via the current session with the next 30 seconds.

<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **POST** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Sites/Transactions/Heartbeat | Yes |

<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |

<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|nb_URL* | string  | IP address of your NetBrain Web API Server  |

<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|statusCode| integer | Code issued by NetBrain server indicating the execution result.<br>790200 for success.<br>792102 if site locked by other operations. |
|statusDescription| string | The explanation of the status code. |

<br><br>

* **Example**

In [2]:
# This API send a hearbeat signal to the server to keep a transaction alive.
def keepAliveSiteTransaction(nb_url, token):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    full_url= nb_url + "/ServicesAPI/API/V1/CMDB/Sites/Transactions/Heartbeat"
    try:
        response = requests.post(full_url, headers=headers, verify=False)
        if response.status_code == 200:
            result = response.json()
            return (result)
        else:
            return ("Site Heartbeat Failed! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
res = keepAliveSiteTransaction(nb_url, token)
res

'Site Heartbeat Failed! - {"statusCode":795003,"statusDescription":"Insufficient permissions: the current user has insufficient permissions to perform the requested operation. The user has no tenant or domain access permission.siteManagement"}'

# Commit Site Transaction API
This API  is used to commit all site change operations since creating a site transaction. Commit operation will automatically trigger site rebuild process.

<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **PUT** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Sites/Transactions | Yes |

<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |

<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|nb_URL* | string  | IP address of your NetBrain Web API Server.  |
|rebuildSite | bool  | Wheather to rebuild site. Optional, false by default.  |

<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|statusCode| integer | Code issued by NetBrain server indicating the execution result.<br>790200 for success.<br>792102 if site locked by other operations. |
|statusDescription| string | The explanation of the status code. |

<br><br>

* **Example**

In [3]:
# This API is used to commit all site change operations since creating a site transaction.
def commitSiteTransaction(nb_url, token, rebuildSite=True):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    full_url= nb_url + "/ServicesAPI/API/V1/CMDB/Sites/Transactions"
    data = {
        "rebuildSite":rebuildSite
    }    
    try:
        response = requests.put(full_url, params=data, headers=headers, verify=False)
        if response.status_code == 200:
            result = response.json()
            return (result)
        else:
            return ("Site commit Failed! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
res = commitSiteTransaction(nb_url, token, rebuildSite=True)
res

'Site commit Failed! - {"statusCode":795003,"statusDescription":"Insufficient permissions: the current user has insufficient permissions to perform the requested operation. The user has no tenant or domain access permission.siteManagement"}'

# Remove Site Transaction API
This API  is used to remove an existing transaction. Note that you need to commit the transaction before calling this API, all uncommitted operations would be lost after delete the transaction.
<br><br>
This API can also be used for rollback purpose.
<br><br>
Removing a site transaction is not necessary since a transaction would automatcially times out after 30 seconds. However, by doing this, you  can give the site control back to the system explicitly within 30 seconds.

<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **DELETE** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Sites/Transactions | Yes |

<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |

<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|nb_URL* | string  | IP address of your NetBrain Web API Server  |

<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|statusCode| integer | Code issued by NetBrain server indicating the execution result.<br>790200 for success.<br>792102 if site locked by other operations. |
|statusDescription| string | The explanation of the status code. |

<br><br>

* **Example**

In [4]:
# This API  is used to remove an existing transaction.
def removeSiteTransaction(nb_url, token):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    full_url= nb_url + "/ServicesAPI/API/V1/CMDB/Sites/Transactions"
    try:
        response = requests.delete(full_url, headers=headers, verify=False)
        if response.status_code == 200:
            result = response.json()
            return (result)
        else:
            return ("Site transaction remove Failed! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
res = removeSiteTransaction(nb_url, token)
res

'Site transaction remove Failed! - {"statusCode":795003,"statusDescription":"Insufficient permissions: the current user has insufficient permissions to perform the requested operation. The user has no tenant or domain access permission.siteManagement"}'

# Create Sites API
This method allow users to create sites with full path names. Note that a new site will be created as a parent site if a site which is been adding doesn't have its parent site show up in current system, 
<br><br>
Note that, this API will replace the ImportSiteTree in 7.0b
<br><br>
Note: this API call needs to be invoked in a site transaction.

<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **Post** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Sites | Yes |

<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |

<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|nb_URL* | string  | IP address of your NetBrain Web API Server.  |
|sites* | List of object  | The list of sites.  |
|sites.sitePath* | string  | Full path name of a site. For example, 'My Network/Site1/Boston'.  |
|sites.isContainer* | bool  | Specify whether the site being added is a container site.  |

<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|statusCode| integer | Code issued by NetBrain server indicating the execution result.<br>790200 for success.<br>792102 if site locked by other operations. |
|statusDescription| string | The explanation of the status code. |

<br><br>

* **Example**

In [5]:
sitePath = "My Network/Site1/Boston"
isContainer = False

# Create sites with full site path.
def createSites(nb_url, token, sitePath, isContainer):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    full_url= nb_url + "/ServicesAPI/API/V1/CMDB/Sites"
    sites=[{"sitePath":sitePath, "isContainer":isContainer}] 
    try:
        response = requests.post(full_url, data=json.dumps({"sites": sites}), headers=headers, verify=False)
        if response.status_code == 200:
            result = response.json()
            return (result)
        else:
            return ("Site Created Failed! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
res = createSites(nb_url, token, sitePath, isContainer)
res

'Site Created Failed! - {"statusCode":795003,"statusDescription":"Insufficient permissions: the current user has insufficient permissions to perform the requested operation. The user has no tenant or domain access permission.siteManagement"}'

# Create a Parent Site API
create a container site. If one parent site doesn't exist in current system, create it before create its child site.
<br><br>
Note: this API call needs to be invoked in a site transaction.

<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **Post** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Sites/Parent | Yes |

<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |

<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|nb_URL* | string  | IP address of your NetBrain Web API Server  |
|sitePath* | string  | Full path name of a site. For example, 'My Network/Site1/Boston'.  |

<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|statusCode| integer | Code issued by NetBrain server indicating the execution result.<br>790200 for success.<br>792102 if site locked by other operations. |
|statusDescription| string | The explanation of the status code. |

<br><br>

* **Example**

In [6]:
# Create parent site with the site path.
def createParentSite(nb_url, token, sitePath):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    full_url= nb_url + "/ServicesAPI/API/V1/CMDB/Sites/Parent"
    # sitePath = "My Network/C0/C1"
    try:
        response = requests.post(full_url, data=json.dumps({"sitePath": sitePath}), headers=headers, verify=False)
        if response.status_code == 200:
            result = response.json()
            return (result)
        else:
            return ("Site Created Failed! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
res = createParentSite(nb_url, token, sitePath)
res

'Site Created Failed! - {"statusCode":795003,"statusDescription":"Insufficient permissions: the current user has insufficient permissions to perform the requested operation. The user has no tenant or domain access permission.siteManagement"}'

# Create a Leaf Site API
create a leaf site. If one parent site doesn't exist in current system, create it before create its child site.
<br><br>
Note: this API call needs to be invoked in a site transaction.

<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **POST** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Sites/Leaf | Yes |

<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |

<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|nb_URL* | string  | IP address of your NetBrain Web API Server  |
|sitePath* | string  | Full path name of a site. For example, 'My Network/Site1/Boston/Dev'.  |

<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|statusCode| integer | Code issued by NetBrain server indicating the execution result.<br>790200 for success.<br>792102 if site locked by other operations. |
|statusDescription| string | The explanation of the status code. |

<br><br>

* **Example**

In [7]:
# Create child site with the site path.
def createChildSite(nb_url, token, sitePath):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    full_url= nb_url + "/ServicesAPI/API/V1/CMDB/Sites/Leaf"
    # sitePath = "My Network/C0/C1"
    try:
        response = requests.post(full_url, data=json.dumps({"sitePath": sitePath}), headers=headers, verify=False)
        if response.status_code == 200:
            result = response.json()
            return (result)
        else:
            return ("Site Created Failed! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
res = createChildSite(nb_url, token, sitePath)
res

'Site Created Failed! - {"statusCode":795003,"statusDescription":"Insufficient permissions: the current user has insufficient permissions to perform the requested operation. The user has no tenant or domain access permission.siteManagement"}'

# Delete a Site API
Delete one site. If a site is a container site, all its child sites of this  site will also be deleted.
<br><br>
Note: this API call needs to be invoked in a site transaction.

<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **DELETE** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Sites/ | Yes |

<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |

<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|nb_URL* | string  | IP address of your NetBrain Web API Server  |
|sitePath* | string  | Full path name of a site. For example, 'My Network/Site1/Boston/Dev'. |
|siteId | string  | Guid of this site to be deleted. Optional. However, parameter must be siteId or sitePath, use siteId if both set. |

<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|statusCode| integer | Code issued by NetBrain server indicating the execution result.<br>790200 for success.<br>792102 if site locked by other operations. |
|statusDescription| string | The explanation of the status code. |

<br><br>

* **Example**

In [8]:
# Create child site with the site path.
def deleteSite(nb_url, token, sitePath):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    full_url= nb_url + "/ServicesAPI/API/V1/CMDB/Sites/"
    # sitePath = "My Network/C0/C1"
    try:
        response = requests.delete(full_url, params={"sitePath": sitePath}, headers=headers, verify=False)
        if response.status_code == 200:
            result = response.json()
            return (result)
        else:
            return ("Site Delete Failed! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
res = deleteSite(nb_url, token, sitePath)
res

'Site Delete Failed! - {"statusCode":795003,"statusDescription":"Insufficient permissions: the current user has insufficient permissions to perform the requested operation. The user has no tenant or domain access permission.siteManagement"}'

# Get Site Info API
This API is used to get the basic information of a site.

<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **GET** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Sites/SiteInfo | Yes |

<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |

<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|nb_URL* | string  | IP address of your NetBrain Web API Server  |
|sitePath* | string  | Full path name of a site. For example, 'My Network/Site1/Boston/Dev'. |
|siteId | string  | Guid of this site to be deleted. Optional. However, parameter must be siteId or sitePath, use siteId if both set. |

<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|siteInfo | object | An object with the basic information of a site.  |
|siteInfo.sitePath | string | Full path of site.  |
|siteInfo.siteId| string | Id of site. This is the only way to get the id of root site. |
|siteInfo.siteType| integer | Type of this site, 0 root site, 1 container site, 2 leaf site.  |
|statusCode| integer | Code issued by NetBrain server indicating the execution result.  |
|statusDescription| string | The explanation of the status code. |

<br><br>

* **Example**

In [9]:
sitePath = "My Network\Americas\Argentina\Provincia de Buenos Aires\Benavidez\AM-ARG-BA-BEN-1621-KM375RAM1618"
siteId = ""

# Get basic information of a site, either siteID or sitePath need to be provided; siteID if both provided.
def getSiteInfo(nb_url, token, siteId, sitePath):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    full_url= nb_url + "/ServicesAPI/API/V1/CMDB/Sites/SiteInfo"
    data = {
        "siteId":siteId,
        "sitePath":sitePath
    }
    try:
        response = requests.get(full_url, params=data, headers=headers, verify=False)
        if response.status_code == 200:
            result = response.json()
            return (result)
        else:
            return ("Get Site Failed! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
res = getSiteInfo(nb_url, token, siteId, sitePath)
res

'Get Site Failed! - {"statusCode":795003,"statusDescription":"Insufficient permissions: the current user has insufficient permissions to perform the requested operation. The user has no tenant or domain access permission.siteManagement"}'

# According to the meeting form, guest role user have the right to call the Get Site Info API. 

# Get Child Sites API
This API is used to get all descedant sites of a container site. Return error if it is a leaf site.

<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **GET** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Sites/ChildSites | Yes |

<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |

<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|nb_URL* | string  | IP address of your NetBrain Web API Server  |
|sitePath* | string  | Full path name of a site. For example, 'My Network/Site1/Boston/Dev'. |
|siteId | string  | Guid of this site to be deleted. Optional. However, parameter must be siteId or sitePath, use siteId if both set. |

<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|sites | list of obeject | A list of all child sites.  |
|sites.siteId| string | Id of site.  |
|sites.sitePath| string | Full path of site. |
|sites.isContainer| bool | If this site is a container site. |
|sites.children| list of string | Id list of immediate child site if this site is a container site. |
|statusCode| integer | Code issued by NetBrain server indicating the execution result.  |
|statusDescription| string | The explanation of the status code. |

<br><br>

* **Example**

In [10]:

# Get all leaf sites of a container site, either siteID or sitePath need to be provided; siteID if both provided.
def getChildSites(nb_url, token, siteId, sitePath):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    full_url= nb_url + "/ServicesAPI/API/V1/CMDB/Sites/ChildSites"
    data = {
        "siteId":siteId,
        "sitePath":sitePath
    }
    try:
        response = requests.get(full_url, params=data, headers=headers, verify=False)
        if response.status_code == 200:
            result = response.json()
            return (result)
        else:
            return ("Get Site Failed! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
res = getChildSites(nb_url, token, siteId, sitePath)
res

'Get Site Failed! - {"statusCode":795003,"statusDescription":"Insufficient permissions: the current user has insufficient permissions to perform the requested operation. The user has no tenant or domain access permission.siteManagement"}'

# Same problem with previous API

# Add Site Devices API
This API is used to add devices that to the site specified by site name. All devices will be marked as manually added type
<br><br>
Note: this API call needs to be invoked in a site transaction.

<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **POST** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Sites/Devices | Yes |

<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |

<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|nb_URL* | string  | IP address of your NetBrain Web API Server.  |
|sitePath* | string  | Full path name of a site. For example, 'My Network/Site1/Boston/Dev'. |
|siteId | string  | Guid of this site to be deleted. Optional. However, parameter must be siteId or sitePath, use siteId if both set. |
|devices* | string[]  | List of device hostnames.  |

<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|statusCode| integer | Code issued by NetBrain server indicating the execution result.<br>790200 for success.<br>792102 if site locked by other operations. |
|statusDescription| string | The explanation of the status code. |

<br><br>

* **Example**

In [13]:
devices = ["R1", "SW1"]
# Add devices into sites
def addSiteDevice(nb_url, token, siteId, sitePath, devices):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    full_url= nb_url + "/ServicesAPI/API/V1/CMDB/Sites/Devices"
    site={
               "siteId":siteId,
               "sitePath":sitePath,
               "devices":devices
        }
    
    try:
        response = requests.post(full_url, data = json.dumps(site), headers=headers, verify=False)
        if response.status_code == 200:
            result = response.json()
            return (result)
        else:
            return("Devices added Fail! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
res = addSiteDevice(nb_url, token, siteId, sitePath, devices)
res

'Devices added Fail! - {"statusCode":795003,"statusDescription":"Insufficient permissions: the current user has insufficient permissions to perform the requested operation. The user has no tenant or domain access permission.siteManagement"}'

# Delete Site Devices API
This API is used to remove devices that  from the site specified by site name. All devices will be  marked as unassigned
<br><br>
Note: this API call needs to be invoked in a site transaction.

<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **DELETE** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Sites/Devices | Yes |

<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |

<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|nb_URL* | string  | IP address of your NetBrain Web API Server.  |
|sitePath* | string  | Full path name of a site. For example, 'My Network/Site1/Boston/Dev'. |
|siteId | string  | Guid of this site to be deleted. Optional. However, parameter must be siteId or sitePath, use siteId if both set. |
|devices* | string[]  | List of device hostnames.  |

<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|statusCode| integer | Code issued by NetBrain server indicating the execution result.<br>790200 for success.<br>792102 if site locked by other operations. |
|statusDescription| string | The explanation of the status code. |

<br><br>

* **Example**

In [14]:

# Delete devices into sites
def deleteSiteDevice(nb_url, token, siteId, sitePath, devices):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    full_url= nb_url + "/ServicesAPI/API/V1/CMDB/Sites/Devices"
    site={
               "siteId":siteId,
               "sitePath":sitePath,
               "devices":devices
        }
    
    try:
        response = requests.delete(full_url, data = json.dumps(site), headers=headers, verify=False)
        if response.status_code == 200:
            result = response.json()
            return (result)
        else:
            return ("Devices removed Fail! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
res = deleteSiteDevice(nb_url, token, siteId, sitePath, devices)
res

'Devices removed Fail! - {"statusCode":795003,"statusDescription":"Insufficient permissions: the current user has insufficient permissions to perform the requested operation. The user has no tenant or domain access permission.siteManagement"}'

# Replace Site Devices API
This API is used to remove all existing devices that  from the site specified by site name and add new devices provided in the devices  parameter. 
<br><br>
Note: this API call needs to be invoked in a site transaction.

<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **PUT** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Sites/Devices | Yes |

<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |

<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|nb_URL* | string  | IP address of your NetBrain Web API Server.  |
|sitePath* | string  | Full path name of a site. For example, 'My Network/Site1/Boston/Dev'. |
|siteId | string  | Guid of this site to be deleted. Optional. However, parameter must be siteId or sitePath, use siteId if both set. |
|devices* | string[]  | List of device hostnames.  |

<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|statusCode| integer | Code issued by NetBrain server indicating the execution result.<br>790200 for success.<br>792102 if site locked by other operations. |
|statusDescription| string | The explanation of the status code. |

<br><br>

* **Example**

In [15]:
# Remove all devices from site transaction, and add new devices to it
def replaceSiteDevice(nb_url, token, siteId, sitePath, devices):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    full_url= nb_url + "/ServicesAPI/API/V1/CMDB/Sites/Devices"
    site={
               "siteId":siteId,
               "sitePath":sitePath,
               "devices":devices
        }
    
    try:
        response = requests.put(full_url, data = json.dumps(site), headers=headers, verify=False)
        if response.status_code == 200:
            result = response.json()
            return (result)
        else:
            return("Device moved Fail! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
res = replaceSiteDevice(nb_url, token, siteId, sitePath, devices)
res

'Device moved Fail! - {"statusCode":795003,"statusDescription":"Insufficient permissions: the current user has insufficient permissions to perform the requested operation. The user has no tenant or domain access permission.siteManagement"}'

# Get Site Devices API
This API is used to get all devices that belong to the site specified by site name.
<br><br>
NOTE:  must be  a leaf site, error would return if the parameter is root site or a container site.

<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **GET** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Sites/Devices | Yes |

<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |

<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|nb_URL* | string  | IP address of your NetBrain Web API Server.  |
|sitePath* | string  | Full path name of a site. For example, 'My Network/Site1/Boston/Dev'. |
|siteId | string  | Guid of this site to be deleted. Optional. However, parameter must be siteId or sitePath, use siteId if both set. |

<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|devices | list of object | A list of all devices.  |
|devices.id| string | Device id.  |
|mgmtIP| string | Management ip of this device. |
|hostname| string | Hostname of this device. |
|deviceTypeName| string | Hostname of this device. |
|statusCode| integer | Code issued by NetBrain server indicating the execution result.  |
|statusDescription| string | The type of the returned device, such as Cisco Router. |

<br><br>

* **Example**

In [16]:
# Get all devices that belong to the site specified by site name, either siteID or sitePath need to be provided; siteID if both provided.
def getSiteDevice(nb_url, token, siteId, sitePath):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    full_url= nb_url + "/ServicesAPI/API/V1/CMDB/Sites/Devices"
    data = {
        "siteId":siteId,
        "sitePath":sitePath
    }
    try:
        response = requests.get(full_url, params=data, headers=headers, verify=False)
        if response.status_code == 200:
            result = response.json()
            return (result)
        else:
            return("Get Site Devices Failed! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
res = getSiteDevice(nb_url, token, siteId, sitePath)
res

'Get Site Devices Failed! - {"statusCode":795003,"statusDescription":"Insufficient permissions: the current user has insufficient permissions to perform the requested operation. The user has no tenant or domain access permission.siteManagement"}'

# According to the meeting form, guest role user have the right to call the Get Site Info API. 